# Python常见高级应用

这部分主要参考[python3-cookbook](https://python3-cookbook.readthedocs.io/zh_CN/latest/index.html)，主要记录一些在实际编写模型算法及开发应用程序的过程中值得一用的高级技巧。

目前主要分为以下部分：

- 数据结构与算法
- 字符串与文本
- 文件与IO
- 类与对象
- 元编程
- 并发编程
- 脚本编程与系统管理
- 测试调试与异常

随着实践的加深，逐渐补充各类高级用法。

## 数据结构与算法

Python 提供了大量的内置数据结构，包括列表，集合以及字典。大多数情况下使用这些数据结构是很简单的。 不过在实际码代码的过程中也会经常碰到到诸如查询，排序和过滤等等这些普遍存在的问题。了解这些基本算法的过程是很有必要的。

首先是排序算法。参考：[排序指南](https://docs.python.org/zh-cn/3.7/howto/sorting.html#)。

Python 列表有一个内置的 list.sort() 方法可以直接修改列表。还有一个 sorted() 内置函数，它会从一个可迭代对象构建一个新的排序列表。

In [2]:
sorted([5, 2, 3, 1, 4])

[1, 2, 3, 4, 5]

In [6]:
a = [5, 2, 3, 1, 4]
a.sort()
a

[1, 2, 3, 4, 5]

给定一个数，寻找一个数组中与该数字最接近的

In [9]:
# Python3 program to find Closest number in a list 
  
def closest(lst, K): 
      
    return lst[min(range(len(lst)), key = lambda i: abs(lst[i]-K))] 
      
# Driver code 
lst = [3.64, 5.2, 9.42, 9.35, 8.5, 8] 
K = 9.1
print(closest(lst, K)) 

9.35


或者使用numpy也可以快速得到

In [10]:
import numpy as np
x = np.arange(100)
print("Original array:")
print(x)
a = np.random.uniform(0,100)
print("Value to compare:")
print(a)
index = (np.abs(x-a)).argmin()
print(x[index])

Original array:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
Value to compare:
37.104596819436466
37


判断一个数组是否递增：

In [5]:
l = range(10000)
print(all(x<y for x, y in zip(l, l[1:])))

True


In [8]:
x=1
y=[x]
type(y)

list

## 字符串与文本

### 字符串中插入变量

想创建一个内嵌变量的字符串，变量被它的值所表示的字符串替换掉。这在包括print结果，构建带参数的url等很多场景下都会用到。

Python并没有对在字符串中简单替换变量值提供直接的支持。 但是通过使用字符串的 format() 方法来解决这个问题。比如：

In [1]:
s = '{name} has {n} messages.'
s.format(name='Guido', n=37)

'Guido has 37 messages.'

或者，如果要被替换的变量能在变量域中找到， 那么可以结合使用format_map() 和 vars() 。就像下面这样：

In [2]:
name = 'Guido'
n = 37
s.format_map(vars())

'Guido has 37 messages.'

vars() 还有一个有意思的特性就是它也适用于对象实例。

In [5]:
class Info:
     def __init__(self, name, n):
            self.name = name
            self.n = n
            
a = Info('Guido',37)
s_out=s.format_map(vars(a))
print(s_out)

Guido has 37 messages.


format 和 format_map() 的一个缺陷就是它们并不能很好的处理变量缺失的情况，一种避免这种错误的方法是另外定义一个含有 __missing__() 方法的字典对象，就像下面这样：

In [7]:
class safesub(dict):
# """防止key找不到"""
    def __missing__(self, key):
        return '{' + key + '}'
    
del n # Make sure n is undefined
s.format_map(safesub(vars()))

'Guido has {n} messages.'

多年以来由于Python缺乏对变量替换的内置支持而导致了各种不同的解决方案。比如常见的%做占位符。不过format() 和 format_map() 相比较上面这些方案而已更加先进，因此应该被优先选择。 使用 format() 方法还有一个好处就是你可以获得对字符串格式化的所有支持(对齐，填充，数字格式化等待)， 而这些特性是使用像模板字符串之类的方案不可能获得的。

In [8]:
s = '{name} has {n} messages，{name}.'
s.format(name='Guido', n=37)

'Guido has 37 messages，Guido.'

## 文件与IO

### 创建临时文件

In [10]:
from tempfile import TemporaryFile

with TemporaryFile('w+t') as f:
    # Read/write to the file
    f.write('Hello World\n')
    f.write('Testing\n')

    # Seek back to beginning and read the data
    f.seek(0)
    data = f.read()

## 类与对象

### 在类中封装属性名

Python程序员**不去依赖语言特性去封装数据**，而是通过**遵循一定的属性和方法命名规约**来达到这个效果。比如：

In [ ]:
class A:
    def __init__(self):
        self._internal = 0 # An internal attribute
        self.public = 1 # A public attribute

    def public_method(self):
        '''
        A public method
        '''
        pass

    def _internal_method(self):
        pass

注意Python并不会真的阻止别人访问内部名称。但是如果你这么做肯定是不好的，可能会导致脆弱的代码。同时还要注意到，使用下划线开头的约定同样适用于模块名和模块级别函数。 内部实现的代码要小心使用。在basic-python中提到过，各类"_"的使用带来的不同，这里重复说明下，有时候可能会遇到在类定义中使用两个下划线(__)开头的命名。比如：

In [ ]:
class B:
    def __init__(self):
        self.__private = 0

    def __private_method(self):
        pass

    def public_method(self):
        pass
        self.__private_method()

这个时候双下划线的名称会在访问它时，被变成其他形式。比如，在前面的类B中，私有属性会被分别**重命名为 _B__private 和 _B__private_method**。 这时候你可能会问这样重命名的目的是什么，答案就是继承——这种属性通过继承是无法被覆盖的。比如：

In [ ]:
class C(B):
    def __init__(self):
        super().__init__()
        self.__private = 1 # Does not override B.__private

    # Does not override B.__private_method()
    def __private_method(self):
        pass

私有名称 __private 和 __private_method 被重命名为 _C __private 和 _C __private_method ，这个跟父类B中的名称是完全不同的。

### 定义接口或抽象基类

定义一个接口或抽象类，并且通过执行类型检查来确保子类实现了某些特定的方法。使用 abc 模块可以很轻松的定义抽象基类。不过目前个人建议在实际编程中还是尽量先规避一些设计模式，看看从流程上能不能简化自己代码的pipeline，这样是更容易的方法，anyway，这里回到接口再看看。抽象类的目的就是让别的类继承它并实现特定的抽象方法：

In [ ]:
from abc import ABCMeta, abstractmethod

class IStream(metaclass=ABCMeta):
    @abstractmethod
    def read(self, maxbytes=-1):
        pass

    @abstractmethod
    def write(self, data):
        pass

In [ ]:
class SocketStream(IStream):
    def read(self, maxbytes=-1):
        pass

    def write(self, data):
        pass

抽象基类的一个主要用途是在代码中检查某些类是否为特定类型，实现了特定接口：

In [ ]:
def serialize(obj, stream):
    if not isinstance(stream, IStream):
        raise TypeError('Expected an IStream')
    pass

## 元编程

软件开发领域中最经典的口头禅就是 **“don’t repeat yourself”**。 也就是说，任何时候当你的程序中存在高度重复(或者是通过剪切复制)的代码时，都应该想想是否有更好的解决方案。在Python当中，通常都可以通过**元编程**来解决这类问题。简而言之，元编程就是关于**创建操作源代码(比如修改、生成或包装原来的代码)的函数和类**。 主要技术是使用**装饰器、类装饰器和元类**。不过还有一些其他技术， 包括**签名对象、使用 exec() 执行代码以及对内部函数和类的反射技术**等。

### 包装器

比如想在函数上添加一个包装器，增加额外的操作处理(比如日志、计时等)。这是很常用的功能。使用额外的代码包装一个函数，可以定义一个装饰器函数：

In [4]:
import time
from functools import wraps

def timethis(func):
    '''
    Decorator that reports the execution time.
    '''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

In [5]:
@timethis
def countdown(n):
#       Counts down
    while n > 0:
        n -= 1

In [6]:
countdown(100000)

countdown 0.01596689224243164


实际上一个装饰器就是一个函数，它接受一个函数作为参数并返回一个新的函数。下面两个函数是等价的：

In [ ]:
@timethis
def countdown(n):
    pass

In [ ]:
def countdown(n):
    pass
countdown = timethis(countdown)

另外，内置的装饰器比如 @staticmethod, @classmethod,@property 原理也是一样的。 

不过在使用装饰器时，要注意复制元信息。即**任何时候**你定义装饰器的时候，都应该使用 functools 库中的 **@wraps 装饰器来注解底层包装函数**。如果你忘记了使用 @wraps ， 那么你会发现被装饰函数丢失了所有有用的信息。

还可以定义带参数的装饰器，比如你想写一个装饰器，给函数添加日志功能，同时允许用户指定日志的级别和其他的选项。 下面是这个装饰器的定义和使用示例：

In [ ]:
from functools import wraps
import logging

def logged(level, name=None, message=None):
    """
    Add logging to a function. level is the logging
    level, name is the logger name, and message is the
    log message. If name and message aren't specified,
    they default to the function's module and name.
    """
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate

# Example use
@logged(logging.DEBUG)
def add(x, y):
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

### 函数重载

python是不支持直接进行函数重载的，不过python允许参数注解。利用它可以简单实现基于类型的方法重载。

In [7]:
class Spam:
    def bar(self, x:int, y:int):
        print('Bar 1:', x, y)

    def bar(self, s:str, n:int = 0):
        print('Bar 2:', s, n)

s = Spam()
s.bar(2, 3) # Prints Bar 1: 2 3
s.bar('hello') # Prints Bar 2: hello 0

Bar 2: 2 3
Bar 2: hello 0


但是对于一般的函数重载会稍微麻烦一些，需要利用一些小技巧，这里给出一个参考资料：[Python 函数如何重载？](https://juejin.im/post/5cbcf38bf265da03af27d327)，就暂时不赘述了。

个人认为既然python不支持，那就尽量避免吧，直接换个名，或者用下if else也不是多大的事情。

## 并发编程

对于并发编程, Python有多种长期支持的方法, 包括多线程, 调用子进程, 以及各种各样的关于生成器函数的技巧.

### 启动与停止线程

为需要并发执行的代码创建/销毁线程：**threading 库**可以**在单独的线程中执行任何的在 Python 中可以调用的对象**。可以创建一个 Thread 对象并将你要执行的对象以 target 参数的形式提供给该对象。

In [3]:
import time
def countdown(n):
    while n > 0:
        print('T-minus', n)
        n -= 1
        time.sleep(1)

# Create and launch a thread
from threading import Thread
t = Thread(target=countdown, args=(3,))
t.start()

T-minus 3
T-minus 2
T-minus 1


当创建好一个线程对象后，该对象**并不会立即执行**，除非你调用它的 **start() 方法**（当你调用 start() 方法时，它会调用你传递进来的函数，并把你传递进来的参数传递给该函数）。Python中的线程会在一个单独的系统级线程中执行（比如说一个 POSIX 线程或者一个 Windows 线程），这些线程将由操作系统来全权管理。线程一旦启动，将独立执行直到目标函数返回。

可以查询一个线程对象的状态，看它是否还在执行：

In [ ]:
if t.is_alive():
    print('Still running')
else:
    print('Completed')

Python解释器直到所有线程都终止前仍保持运行。对于需要长时间运行的线程或者需要一直运行的后台任务，你应当考虑使用后台线程。
```python
# 使用daemon
t = Thread(target=countdown, args=(10,), daemon=True)
t.start()
```

由于全局解释锁（GIL）的原因，Python 的线程被限制到同一时刻只允许一个线程执行这样一个执行模型。所以，Python 的线程更适用于处理I/O和其他需要并发执行的阻塞操作（比如等待I/O、等待从数据库获取数据等等），而**不是需要多处理器并行的计算密集型任务**。所以这块暂时不需要太多关注。

### 简单并行编程

有个程序要执行CPU密集型工作，想让他利用**多核CPU**的优势来运行的快一点。

concurrent.futures 库提供了一个 ProcessPoolExecutor 类， 可被用来在一个单独的Python解释器中执行计算密集型函数。 不过，要使用它，首先要有一些计算密集型的任务。一个脚本，在这些日志文件中查找出所有访问过robots.txt文件的主机(因为我没有日志文件，所以没运行结果)

In [4]:
# findrobots.py

import gzip
import io
import glob

def find_robots(filename):
    '''
    Find all of the hosts that access robots.txt in a single log file
    '''
    robots = set()
    with gzip.open(filename) as f:
        for line in io.TextIOWrapper(f,encoding='ascii'):
            fields = line.split()
            if fields[6] == '/robots.txt':
                robots.add(fields[0])
    return robots

def find_all_robots(logdir):
    '''
    Find all hosts across and entire sequence of files
    '''
    files = glob.glob(logdir+'/*.log.gz')
    all_robots = set()
    for robots in map(find_robots, files):
        all_robots.update(robots)
    return all_robots

if __name__ == '__main__':
    robots = find_all_robots('logs')
    for ipaddr in robots:
        print(ipaddr)

前面的程序使用了通常的**map-reduce风格**来编写。 函数 find_robots() 在一个文件名集合上做map操作，并将结果汇总为一个单独的结果， 也就是 find_all_robots() 函数中的 all_robots 集合。 现在，假设你想要修改这个程序让它使用多核CPU。 很简单——只需要**将map()操作替换为一个 concurrent.futures 库中生成的类似操作即可**。

In [ ]:
# findrobots.py

import gzip
import io
import glob
from concurrent import futures

def find_robots(filename):
    '''
    Find all of the hosts that access robots.txt in a single log file

    '''
    robots = set()
    with gzip.open(filename) as f:
        for line in io.TextIOWrapper(f,encoding='ascii'):
            fields = line.split()
            if fields[6] == '/robots.txt':
                robots.add(fields[0])
    return robots

def find_all_robots(logdir):
    '''
    Find all hosts across and entire sequence of files
    '''
    files = glob.glob(logdir+'/*.log.gz')
    all_robots = set()
    with futures.ProcessPoolExecutor() as pool:
        for robots in pool.map(find_robots, files):
            all_robots.update(robots)
    return all_robots

if __name__ == '__main__':
    robots = find_all_robots('logs')
    for ipaddr in robots:
        print(ipaddr)

## 脚本编程与系统管理

首先补充一些基本的在python脚本中执行系统命令的代码。cank 

In [2]:
import os
os.system('ls')

0


In [3]:
tmp = os.popen('ls *.py').readlines()
tmp

['mydict2.py\n', 'mydict.py\n', 'mydict_test.py\n']

In [4]:
import subprocess

p = subprocess.Popen('ls', shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
for line in p.stdout.readlines():
    print (line)
retval = p.wait()

b'1-basic-python.ipynb\n'
b'2-advanced-python.ipynb\n'
b'app.log\n'
b'config.ini\n'
b'data.json\n'
b'logconfig.ini\n'
b'mydict2.py\n'
b'mydict.py\n'
b'mydict_test.py\n'
b'__pycache__\n'
b'test1.txt\n'
b'test.txt\n'


### 命令行解析器

Python 命令行与参数解析方法有很多工具，这里参考python[官方文档](https://docs.python.org/zh-cn/3/library/argparse.html)
和[blog](https://zhuanlan.zhihu.com/p/34395749)学习使用python 自带的argparse ，来说明python 如何进行命令行解析。

通俗来说，命令行与参数解析就是当你输入cmd 打开dos 交互界面时候，启动程序要进行的参数给定。比如在dos 界面输入：

```code
python openPythonFile.py "a" -b "number"
```

其中，"a" -b等就是命令行与参数解析要做的事情。

就是设计程序在运行时必须给定某些额外参数才能运行，也就是如果设置了命令行参数解析，那么各种编译器按F5 是无法直接运行程序的。

用途就是不能随便就能运行脚本，可以达到一定程度上的安全功能。

总体上分为三大步：

- 创建解析
- 添加参数
- 解析参数

使用 argparse 的第一步是创建一个 ArgumentParser 对象。

ArgumentParser 对象包含将命令行解析成 Python 数据类型所需的全部信息。

In [ ]:
import argparse
parser = argparse.ArgumentParser(description='Process some integers.')

第二步是添加参数，给一个 ArgumentParser 添加程序参数信息是通过调用 add_argument() 方法完成的。

通常，这些调用指定 ArgumentParser 如何获取命令行字符串并将其转换为对象。这些信息在 parse_args() 调用时被存储和使用。

add_argument方法定义单个的命令行参数应当如何解析。每个形参都在下面有它自己更多的描述：

- name or flags - 一个命名或者一个选项字符串的列表，例如 foo 或 -f, --foo。
- action - 当参数在命令行中出现时使用的动作基本类型。
- nargs - 命令行参数应当消耗的数目。
- const - 被一些 action 和 nargs 选择所需求的常数。
- default - 当参数未在命令行中出现时使用的值。
- type - 命令行参数应当被转换成的类型。
- choices - 可用的参数的容器。
- required - 此命令行选项是否可省略 （仅选项可用）。
- help - 一个此选项作用的简单描述。
- metavar - 在使用方法消息中使用的参数值示例。
- dest - 被添加到 parse_args() 所返回对象上的属性名。大多数ArgumentParser actions增加一些值作为the object的一个属性被parse_args()返回 

In [ ]:
parser.add_argument('integers', metavar='N', type=int,
                    nargs='+', help='an integer for the accumulator')
parser.add_argument('--sum', dest='accumulate', action='store_const',
                    const=sum, default=max,
                    help='sum the integers (default: find the max)')

最后ArgumentParser 通过 parse_args() 方法解析参数。它将检查命令行，把每个参数转换为适当的类型然后调用相应的操作。

在大多数情况下，这意味着一个简单的 Namespace 对象将从命令行参数中解析出的属性构建。

在脚本中，通常 parse_args() 会被不带参数调用，而 ArgumentParser 将自动从 sys.argv 中确定命令行参数。

In [ ]:
args=parser.parse_args(['--sum', '7', '-1', '42'])
print(args.accumulate(args.integers))

### 读取配置文件

很多情况下，我们需要通过配置文件来定义一些参数性质的数据，因为配置文件作为一种可读性很好的格式，非常适用于存储程序中的配置数据。 

在每个配置文件中，配置数据会被分组（比如例子中的“installation”、 “debug” 和 “server”）。 每个分组在其中指定对应的各个变量值。那么如何读取普通.ini格式的配置文件？

在python中，configparser 模块能被用来读取配置文件。例如，假设有配置文件config.ini。下面给出读取代码：

In [1]:
from configparser import ConfigParser
cfg = ConfigParser()
cfg.read('config.ini')

['config.ini']

In [2]:
cfg.sections()

['installation', 'debug', 'server']

In [3]:
cfg.get('installation','library')

'/usr/local/lib'

In [4]:
cfg.getboolean('debug','log_errors')

True

In [5]:
cfg.getint('server','port')

8080

In [6]:
cfg.getint('server','nworkers')

32

In [7]:
print(cfg.get('server','signature'))


Brought to you by the Python Cookbook


还可以读取一个section下的所有keys或所有键值对，参考：[Python 读取写入配置文件 —— ConfigParser](https://blog.csdn.net/jiede1/article/details/79064780)

In [9]:
cfg.options("installation") 

['library', 'include', 'bin', 'prefix']

In [10]:
cfg.items("installation")  

[('library', '/usr/local/lib'),
 ('include', '/usr/local/include'),
 ('bin', '/usr/local/bin'),
 ('prefix', '/usr/local')]

如果需要，还能修改配置并使用 cfg.write() 方法将其写回到文件中。例如：

In [8]:
cfg.set('server','port','9000')
cfg.set('debug','log_errors','False')
import sys
cfg.write(sys.stdout)

[installation]
library = %(prefix)s/lib
include = %(prefix)s/include
bin = %(prefix)s/bin
prefix = /usr/local

[debug]
log_errors = False
show_warnings = False

[server]
port = 9000
nworkers = 32
pid-file = /tmp/spam.pid
root = /www/root
signature = 
	Brought to you by the Python Cookbook



### 日志功能

即在脚本和程序中将诊断信息写入日志文件。打印日志最简单方式是使用 logging 模块。

In [16]:
import logging
import logging.config

def main():
    # Configure the logging system
#     logging.basicConfig(
#         filename='app.log',
#         level=logging.ERROR
#     )
    logging.config.fileConfig('logconfig.ini')

    # Variables (to make the calls that follow work)
    hostname = 'www.python.org'
    item = 'spam'
    filename = 'data.csv'
    mode = 'r'

    # Example logging calls (insert into your program)
    logging.critical('Host %s unknown', hostname)
    logging.error("Couldn't find %r", item)
    logging.warning('Feature is deprecated')
    logging.info('Opening file %r, mode=%r', filename, mode)
    logging.debug('Got here')

if __name__ == '__main__':
    main()

上面五个日志调用（critical(), error(), warning(), info(), debug()）以降序方式表示不同的严重级别。 basicConfig() 的 level 参数是一个过滤器。 所有级别低于此级别的日志消息都会被忽略掉。 每个logging操作的参数是一个消息字符串，后面再跟一个或多个参数。 构造最终的日志消息的时候我们使用了%操作符来格式化消息字符串。可以将日志配置写入配置文件，然后调用，是一样的，代码如上所示。

## 测试、调试与异常

在Python测试代码之前没有编译器来分析代码，因此使得测试成为开发的一个重要部分。这里记录一些关于测试、调试和异常处理的常见问题。

### 关于单元测试

之前basic部分已经记录了一些unittest内容，这里补充一些诸如mock等概念的基本内容。主要参考了：[Python Mocking, You Are A Tricksy Beast](https://medium.com/python-pandemonium/python-mocking-you-are-a-tricksy-beast-6c4a1f8d19b2)，[An Introduction to Mocking in Python](https://www.toptal.com/python/an-introduction-to-mocking-in-python)和[Understanding the Python Mock Object Library](https://realpython.com/python-mock-library/#patch-as-a-decorator)。

#### 为什么要使用mock

测试是验证逻辑是否正确的可靠高效的方式。不过由于一些复杂逻辑和依赖库，会使得测试变得困难。一个使用Python mock 对象的理由就是在测试过程中控制代码的行为。比如代码发送HTTP请求到外部服务，只有当服务的行为符合您的预期时，您的测试才会可预测地执行。有时，这些外部服务行为的临时更改可能导致测试套件中的间歇性故障。因此，我们想要使我们的代码在一个受控的环境下测试。而使用mock对象可以做到这一点。

有时，很难测试代码的某些环节，比如except代码块，if代码块，因为可能不出现这样的场景，这时候使用mock对象也可以帮助控制代码执行的路径来使程序能运行到这些地方，提升code coverage。

另一个原因是更好地理解如何使用代码的真实副本。一个python mock对象包含关于其用法的数据，您可以检查这些数据，比如：是否调用了某个方法，如何调用某个方法，多久一次调用某个方法。

此外，有时候我们会面临这样的情形，即我们想测试我们的代码，但是不想产生一些脏结果，比如：我们想要测试facebook的上传功能，但是并不想真的上传一个内容上去。再比如，写一个弹出一个CD drive的脚本，或者一个从/tmp文件夹清除缓存的服务，或者一个绑定到TCP端口的socket服务，这些在unittest下都会产生dirty结果。作为写代码的，更关心的是您的库成功地调用了系统函数来弹出CD，而不是每次运行测试时都还需要打开CD drive。保持单元测试的效率和性能意味着尽量避免运行自动化测试的缓慢代码。

还有，个人认为，在实际测试算法代码的过程中，后面函数会用到前面过程的数据结果，如果每次都从头测试，那么花费时间会很长，因此保存中间计算结果，然后使用mock来代替前面的函数过程，直接读取中间结果来供后面代码测试也是十分必要的。

而unittest.mock可以客服这些困难。接下来就看看mock究竟是什么。

#### What Is Mocking?

mock就是“看起来像真的”的意思，在**测试环境下**，一个mock对象**代替模拟**一个真实的对象。是一个灵活有力的提升测试质量的工具。

unittest.mock库提供了一个叫做Mock的类，可以使用它来模拟代码中的真实对象。Mock还提供了一个函数patch()，它用Mock实例提到了代码中的真实对象。可以将patch()用为decorator，也可以用作context manager ， 取决于想要模拟的对象控制的scope。一旦退出指定的scope，patch就会立刻用真实的副本来取代mock对象。

首先先看看Mock。

In [1]:
from unittest.mock import Mock
mock = Mock()
mock

<Mock id='140398994456464'>

现在就可以使用 Mock来替代代码中的对象了。可以传递它为一个函数的参数或者重定义一个对象。形如：

```python
# Pass mock as an argument to do_something()
do_something(mock)

# Patch the json library
json = mock
```

注意，当你替换一个对象时，Mock必须要看起来真的像这个对象。比如要mock json库，那么程序调用dumps函数，你的mock对象里必须得有一个dumps函数。

In [2]:
mock.some_attribute

<Mock name='mock.some_attribute' id='140398994419280'>

In [3]:
mock.do_something()

<Mock name='mock.do_something()' id='140398993929552'>

Mock可以创建任意属性，可以代替任意对象。用一下之前提到的json例子：

In [5]:
json = Mock()
json.dumps()

<Mock name='mock.dumps()' id='140399003271056'>

可以看到很容易的就mock了kson库和其dumps函数，dumps可以接受任意参数，返回值也是一个mock对象，因此mock可以用到很复杂的环境下。很灵活。

接下来，看看如何用mock更好地理解代码。Mock实例存储这怎么使用它们的数据。

首先可以断言程序使用了你期望的一个对象。

In [1]:
from unittest.mock import Mock
json = Mock()
json.loads('{"key": "value"}')

<Mock name='mock.loads()' id='140186597181904'>

In [2]:
json.loads.assert_called()

In [3]:
json.loads.assert_called_with('{"key": "value"}')

In [4]:
json.loads.assert_called_once_with('{"key": "value"}')

In [5]:
json.loads('{"key": "value"}')

<Mock name='mock.loads()' id='140186597181904'>

In [6]:
json.loads.assert_called_once()

AssertionError: Expected 'loads' to have been called once. Called 2 times.

In [7]:
json.loads.assert_called_once_with('{"key": "value"}')

AssertionError: Expected 'loads' to be called once. Called 2 times.

In [8]:
json.loads.assert_not_called()

AssertionError: Expected 'loads' to not have been called. Called 2 times.

.assert_called()函数确保了调用mocked函数。 .assert_called_once()可以检查调用的次数。

第二，可以查看特殊属性以理解应用是如何使用该对象的。

In [9]:
from unittest.mock import Mock
json = Mock()
json.loads('{"key": "value"}')

<Mock name='mock.loads()' id='140186595899664'>

In [10]:
json.loads.call_count

1

In [11]:
json.loads.call_args

call('{"key": "value"}')

In [12]:
json.loads.call_args_list

[call('{"key": "value"}')]

In [13]:
json.method_calls

[call.loads('{"key": "value"}')]

通过以上测试代码可以使用各类属性来保证对象行为是想要的。这是一些固有的方法，接下来看看如何定制mocked方法。

管理一个Mock的返回值。一个使用mocks的原因就是控制代码的行为。一种十分常用的方式就是指定一个函数的返回值。

首先，创建一个文件my_calendar.py，代码见文件。然后执行下列语句：

In [14]:
!python my_calendar.py

上述代码如果在周末的时候运行是会报错的。而平常是正确的。写测试代码时候，很重要的是确保结果是可预测的。可以使用Mock来去除代码中的不确定性。如下所示，通过Mock .today() 指定返回值来实现。

In [15]:
import datetime
from unittest.mock import Mock

# Save a couple of test days
tuesday = datetime.datetime(year=2019, month=1, day=1)
saturday = datetime.datetime(year=2019, month=1, day=5)

# Mock datetime to control today's date
datetime = Mock()

def is_weekday():
    today = datetime.datetime.today()
    # Python's datetime library treats Monday as 0 and Sunday as 6
    return (0 <= today.weekday() < 5)

# Mock .today() to return Tuesday
datetime.datetime.today.return_value = tuesday
# Test Tuesday is a weekday
assert is_weekday()
# Mock .today() to return Saturday
datetime.datetime.today.return_value = saturday
# Test Saturday is not a weekday
assert not is_weekday()

#### patch()

前面已经提到，unittest.mock还有一个很好的机制：patch(), 装饰器，补丁。

接下来通过实例分析。

看下mock官方的说明：“mock is a library for testing in Python. It allows you to replace parts of your system under test with mock objects and make assertions about how they have been used.”

关键是如何理解“replace parts of your system”。这里的parts指的是什么。实际上可以包括：

- functions
- classes，objects

可以使用Mock 或 MagicMock 类的实例的“mock 对象”来替代它们。

比如mocking functions，以一个简单的函数为例，文件 simple.py。在测试中调用该函数有两种方式，其一是直接使用：

In [8]:
import simple
from unittest import mock
def use_simple_function():
    result = simple.simple_function()
    print(result)
use_simple_function()

You have called simple_function


其二就是利用mock进行测试，为了模仿 simple_function，可以使用 mock.patch decorator。该decorator可以使用户通过以‘package.module.FunctionName’形式传入字符串参数指定想要mock的内容。对于本例，即module simple和函数simple_function，decorator如下第一行，函数可以表达为如下所示。其中函数mock_simple_function的参数是MagicMock类对象，用来代替想要mock的函数。

In [9]:
@mock.patch('simple.simple_function')
def mock_simple_function(mock_simple_func):
    print(mock_simple_func)
mock_simple_function()

<MagicMock name='simple_function' id='140138476026896'>


通过语句“@mock.patch(‘simple.simple_function’)”表明想使用MagicMock对象表达来替代simple_function，这个对象放入了mock_simple_func这一函数形参中。如上代码执行结果所示，输出是一个MagicMock对象，就是它替代了simple_function被调用。可从以下代码中看出：

In [6]:
@mock.patch('simple.simple_function')
def mock_simple_function(mock_simple_func):
    print(mock_simple_func)
    print(simple.simple_function)
    result = simple.simple_function()
    print(result)
mock_simple_function()

<MagicMock name='simple_function' id='140716760885328'>
<MagicMock name='simple_function' id='140716760885328'>
<MagicMock name='simple_function()' id='140716760928784'>


不过现在还有个更重要的问题：为什么创建新的MagicMock对象，究竟怎么调用这个对象。所以要看看MagicMock这个类。MagicMock之所以叫magic，是因为它有大多数python的magic函数的默认实现，即那些名称前后有双下划线的函数，可以查看[这里](http://www.ironpythoninaction.com/magic-methods.html)。比如__call__，即可以让一个类对象可以像函数那样被调用。因此，MagicMock对象是可以直接像函数那样被调用的。

回到例子中，现在已经mock了simple_function，但是还没有使用它来做什么。那么现在想返回simple_function()的结果要怎么做呢？可以使用MagicMock的return_value属性来实现：

In [13]:
@mock.patch('simple.simple_function')
def mock_simple_function(mock_simple_func):
    mock_simple_func.return_value = "You have mocked simple_function"
    result = simple.simple_function()
    print(result)
mock_simple_function()

You have mocked simple_function


从上面结果可以看出很好地模仿了simple_function函数结果。

如果除了返回值之外，还想其他功能，可以使用MagicMock.side_effect 。比如想要测试一个错误并抛出一个异常。

In [17]:
def side_effect_function():
    raise FloatingPointError("A disastrous floating point error has occurred")

@mock.patch('simple.simple_function')
def mock_simple_function_with_side_effect(mock_simple_func):
    mock_simple_func.side_effect = side_effect_function
    result = simple.simple_function()
    print(result)
    
mock_simple_function_with_side_effect()

FloatingPointError: A disastrous floating point error has occurred

接下来看一看如何mock类。在simple.py 文件中定义一个类。然后定义一个调用的函数，首先还是传统的调用方式：

In [1]:
import simple
def use_simple_class():
    inst = simple.SimpleClass()
    print(inst.explode())
use_simple_class()

KABOOM!


然后接下来看看mock下如何操作。依然使用@mock.patch decorator

In [3]:
from unittest import mock
@mock.patch("simple.SimpleClass")
def mock_simple_class(mock_class):
    print(mock_class)
mock_simple_class()

<MagicMock name='SimpleClass' id='140716761038736'>


通过@mock.patch decorator ，参数mock_class使用了MagicMock对象来代替了SimpleClass对象。

In [4]:
@mock.patch("simple.SimpleClass")
def mock_simple_class(mock_class):
    print(mock_class)
    print(simple.SimpleClass)
mock_simple_class()

<MagicMock name='SimpleClass' id='140716760823376'>
<MagicMock name='SimpleClass' id='140716760823376'>


接下来创建一个SimpleClass实例，然后打印，看看会发生什么。

In [5]:
@mock.patch("simple.SimpleClass")
def mock_simple_class(mock_class):
    print(mock_class)
    print(simple.SimpleClass)
    inst = simple.SimpleClass()
    print(inst)
mock_simple_class()

<MagicMock name='SimpleClass' id='140716761247504'>
<MagicMock name='SimpleClass' id='140716761247504'>
<MagicMock name='SimpleClass()' id='140716760859536'>


可以看出，调用SimpleClass() 就调用了MagicMock对象作为函数来创建了MagicMock对象。

到这里，mock一个函数和mock一个类并没有什么区别。不过在类中，使用的更多是其对象。从下面的例子中可以看出，类的MagicMock对象返回值是类对象。

简单小结一下，就是mock一个class时创建了一个MagicMock对象。创建一个类对象时，新的MagicMock对象也被创建，另外类的MagicMock对象返回值也就是类对象的MagicMock对象。

In [7]:
@mock.patch("simple.SimpleClass")
def mock_simple_class(mock_class):
    print(mock_class)
    print(simple.SimpleClass)
    inst = simple.SimpleClass()
    print(inst)
    print(mock_class.return_value)
mock_simple_class()

<MagicMock name='SimpleClass' id='140716760963344'>
<MagicMock name='SimpleClass' id='140716760963344'>
<MagicMock name='SimpleClass()' id='140716761002704'>
<MagicMock name='SimpleClass()' id='140716761002704'>


此外，可以在类对象中通过explode函数来设置return_value，以mock 类对象的返回值。

In [8]:
@mock.patch("simple.SimpleClass")
def mock_simple_class(mock_class):
    mock_class.return_value.explode.return_value = "BOO!"
    inst = simple.SimpleClass()
    result = inst.explode()
    print(result)
    print(mock_class.return_value)
mock_simple_class()

BOO!
<MagicMock name='SimpleClass()' id='140716760511824'>


### 给程序性能测试

测试程序运行所花费的时间并做性能测试。如果只是简单的想测试下程序整体花费的时间， 通常使用Unix时间函数就行了，比如：

```code
bash % time python3 someprogram.py
real 0m13.937s
user 0m12.162s
sys  0m0.098s
bash %
```

如果你还需要一个程序各个细节的详细报告，可以使用 cProfile 模块：

```code
bash % python3 -m cProfile someprogram.py
bash %
```

不过通常情况是介于这两个极端之间。比如已经知道代码运行时在少数几个函数中花费了绝大部分时间。 对于这些函数的性能测试，可以使用一个简单的装饰器。要使用这个装饰器，只需要将其放置在要进行性能测试的函数定义前即可。

In [1]:
# timethis.py

import time
from functools import wraps

def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        r = func(*args, **kwargs)
        end = time.perf_counter()
        print('{}.{} : {}'.format(func.__module__, func.__name__, end - start))
        return r
    return wrapper

@timethis
def countdown(n):
     while n > 0:
            n -= 1

countdown(10000000)

__main__.countdown : 0.43558346499776235


要测试某个代码块运行时间，你可以定义一个上下文管理器

In [ ]:
@contextmanager
def timeblock(label):
    start = time.perf_counter()
    try:
        yield
    finally:
        end = time.perf_counter()
        print('{} : {}'.format(label, end - start))
        
with timeblock('counting'):
     n = 10000000
    while n > 0:
             n -= 1

In [ ]:
对于测试很小的代码片段运行性能，使用 timeit 模块会很方便

In [2]:
from timeit import timeit
timeit('math.sqrt(2)', 'import math')
timeit('sqrt(2)', 'from math import sqrt')
timeit('math.sqrt(2)', 'import math', number=10000000)
timeit('sqrt(2)', 'from math import sqrt', number=10000000)

0.4633127580018481

当执行性能测试的时候，需要注意的是你获取的结果都是近似值。 time.perf_counter() 函数会在给定平台上获取最高精度的计时值。 不过，它仍然还是基于时钟时间，很多因素会影响到它的精确度，比如机器负载。 如果你对于执行时间更感兴趣，使用 time.process_time() 来代替它。

In [ ]:
from functools import wraps
def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.process_time()
        r = func(*args, **kwargs)
        end = time.process_time()
        print('{}.{} : {}'.format(func.__module__, func.__name__, end - start))
        return r
    return wrapper

### 加速程序运行

程序运行太慢，想在不使用复杂技术比如C扩展或JIT编译器的情况下加快程序运行速度。

关于程序优化的第一个准则是“不要优化”，第二个准则是“不要优化那些无关紧要的部分”。 如果你的程序运行缓慢，首先得对它进行性能测试找到问题所在。

通常来讲会发现程序在少数几个热点地方花费了大量时间， 比如内存的数据处理循环。一旦定位到这些点，就可以使用下面这些实用技术来加速程序运行。

#### 使用函数

很多程序员刚开始会使用Python语言写一些简单脚本。 当编写脚本的时候，通常习惯了写毫无结构的代码。比如：

```python
# somescript.py

import sys
import csv

with open(sys.argv[1]) as f:
     for row in csv.reader(f):

         # Some kind of processing
         pass
```

像这样定义在全局范围的代码运行起来要比定义在函数中运行慢的多。 这种速度差异是由于局部变量和全局变量的实现方式（**使用局部变量要更快些**）。 因此，如果想让程序运行更快些，只需要将脚本语句放入函数中即可：

```python
# somescript.py
import sys
import csv

def main(filename):
    with open(filename) as f:
         for row in csv.reader(f):
             # Some kind of processing
             pass

main(sys.argv[1])
```

根据经验，使用函数带来15-30%的性能提升是很常见的。

局部变量会比全局变量运行速度快。 对于频繁访问的名称，通过将这些名称变成局部变量可以加速程序运行。

对于类中的属性访问也同样适用于这个原理。 通常来讲，查找某个值比如 self.name 会比访问一个局部变量要慢一些。 在内部循环中，可以将某个需要频繁访问的属性放入到一个局部变量中。

#### 尽可能去掉属性访问

每一次**使用点(.)操作符来访问属性的时候会带来额外的开销**。 它会触发特定的方法，比如 __getattribute__() 和 __getattr__() ，这些方法会进行字典操作操作。

通常你可以使用 from module import name 这样的导入形式，以及使用绑定的方法。

In [1]:
import math

def compute_roots(nums):
    result = []
    for n in nums:
        result.append(math.sqrt(n))
    return result

# Test
nums = range(1000000)
for n in range(100):
    r = compute_roots(nums)

修改compute_roots函数：

In [ ]:
from math import sqrt

def compute_roots(nums):

    result = []
    result_append = result.append
    for n in nums:
        result_append(sqrt(n))
    return result

修改后的版本运行时间会减少一些。唯一不同之处就是消除了属性访问。 用 sqrt() 代替了 math.sqrt() 。 The result.append() 方法被赋给一个局部变量 result_append ，然后在内部循环中使用它。

这些改变只有在大量重复代码中才有意义，比如循环。 因此，这些优化也只是在某些特定地方才应该被使用。

#### 避免不必要的抽象

任何时候当你使用额外的处理层（比如装饰器、属性访问、描述器）去包装你的代码时，都会让程序运行变慢。

In [2]:
class A:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    @property
    def y(self):
        return self._y
    @y.setter
    def y(self, value):
        self._y = value
        
from timeit import timeit
a = A(1,2)
timeit('a.x', 'from __main__ import a')

0.037184744999876784

In [3]:
timeit('a.y', 'from __main__ import a')

0.11615511300010439

访问属性y相比属性x而言慢的不止一点点，大概慢了4.5倍。 如果你在意性能的话，那么就需要重新审视下对于y的属性访问器的定义是否真的有必要了。 如果没有必要，就使用简单属性吧。 如果仅仅是因为其他编程语言需要使用getter/setter函数就去修改代码风格，这个真的没有必要。

#### 使用内置的容器

内置的数据类型比如字符串、元组、列表、集合和字典都是使用C来实现的，运行起来非常快。 如果想自己实现新的数据结构（比如链接列表、平衡树等）， 那么要想在性能上达到内置的速度几乎不可能，因此，还是乖乖的使用内置的吧。

另外，还要避免创建不必要的数据结构或复制。

#### 并行编程

这部分有参考：[Python性能优化的20条建议](https://segmentfault.com/a/1190000000666603)。

可以通过内置的模块multiprocessing实现下面几种并行模式：

多进程：对于CPU密集型的程序，可以使用multiprocessing的Process,Pool等封装好的类，通过多进程的方式实现并行计算。但是因为进程中的通信成本比较大，对于进程之间需要大量数据交互的程序效率未必有大的提高。

多线程：对于IO密集型的程序，multiprocessing.dummy模块使用multiprocessing的接口封装threading，使得多线程编程也变得非常轻松(比如可以使用Pool的map接口，简洁高效)。

分布式：multiprocessing中的Managers类提供了可以在不同进程之共享数据的方式，可以在此基础上开发出分布式的程序。

不同的业务场景可以选择其中的一种或几种的组合实现程序性能的优化。

#### 讨论

**在优化之前，有必要先研究下使用的算法**。 选择一个复杂度为 O(n log n) 的算法要比你去调整一个复杂度为 O(n**2) 的算法所带来的性能提升要大得多。

如果你觉得你还是得进行优化，那么请从整体考虑。 作为一般准则，不要对程序的每一个部分都去优化,因为这些修改会导致代码难以阅读和理解。 你应该**专注于优化产生性能瓶颈的地方，比如内部循环**。

对循环的优化所遵循的原则是尽量减少循环过程中的计算量，有多重循环的尽量将内层的计算提到上一层———[Python 代码性能优化技巧](https://www.ibm.com/developerworks/cn/linux/l-cn-python-optim/index.html)。

这里对循环做些补充，参考：[Python性能诀窍](http://pfmiles.github.io/blog/python-speed-performance-tips/).

Python支持好几种循环结构。for语句是最常用的。它遍历一个序列的每个元素，将每个元素赋值给循环变量。如果你的循环体很简单，for循环本身的解释成本将占据大部分的开销。这个时候**map函数**就能派上用场了。你可以将map函数看作是for循环采用C代码来实现。唯一的约束是“**循环体”必须是一个函数调用**。**list comprehension 列表生成式**除了语法上的便利性之外，他们常常和等价的map调用一样快甚至更快。比如：

In [ ]:
newlist = []
for word in oldlist:
    newlist.append(word.upper())

可以使用map函数将这个循环由解释执行推到编译好的C代码中去执行：

In [ ]:
newlist = map(str.upper, oldlist)

List comprehension在python 2.0的时候被加入。它们提供了一种更紧凑的语法和更高效的方式来表达上面的for循环：

In [ ]:
newlist = [s.upper() for s in oldlist]

如果优化要求比较高，本节的这些简单技术满足不了，那么可以研究下基于即时编译（JIT）技术的一些工具。 例如，PyPy工程是Python解释器的另外一种实现，它会分析程序运行并对那些频繁执行的部分生成本机机器码。 它有时候能极大的提升性能，通常可以接近C代码的速度。 不过可惜的是，PyPy还不能完全支持Python3.。

还可以考虑下Numba工程， Numba是一个在你使用装饰器来选择Python函数进行优化时的动态编译器。 这些函数会使用LLVM被编译成本地机器码。它同样可以极大的提升性能。 但是，跟PyPy一样，它对于Python 3的支持现在还停留在实验阶段。

最后引用John Ousterhout说过的话作为结尾：“最好的性能优化是从不工作到工作状态的迁移”。 直到你真的需要优化的时候再去考虑它。确保你程序正确的运行通常比让它运行更快要更重要一些（至少开始是这样的）.